In [1]:
import requests
import pandas as pd
import numpy as np
import nbconvert
import datetime
import warnings; warnings.simplefilter('ignore')
import plotly.graph_objects as go
from IPython.display import display, Markdown, Image, SVG
import re
import bmondata
from bmondata import Server
from dateutil.relativedelta import relativedelta
import urllib
import io
import requests
import math


In [2]:

def get_fuel_df(file_path):
    ak_energy_url = 'http://ak-energy-data.analysisnorth.com/'
    """Returns a Pandas DataFrame that is found at the 'file_path'
    below the Base URL for accessing data.  The 'file_path' should end
    with '.pkl' and points to a pickled, compressed (bz2), Pandas DataFrame.
    """
    b = requests.get(urllib.parse.urljoin(ak_energy_url, file_path)).content
    df = pd.read_pickle(io.BytesIO(b), compression='bz2')
    return df

In [3]:
heat_flow_sensor_dict = {
    'Quinhagak': 'quinhagak_hr_heat_rate'
}

total_heat_sensor_dict = {
    'Quinhagak': 'quinhagak_hr_total_heat'
}

fuel_price_Dict = {
    'Quinhagak': 6.13
}

ak_warm_city_building_dict= {
    'Quinhagak': 'Quinhagak'
}

In [4]:
# Papermill variables to change to look at different buildings / organizations 
building_id = 32
server_web_address = 'http://rm.anthc.webfactional.com'



In [5]:
averaging_time_length = 24 #average data over 1 day timespan
kbtu_per_gallon_oil = 134
boiler_efficiency = .75 

In [6]:
server = Server(server_web_address)
building_df = server.buildings(building_id)
current_building_name = building_df[0]['title']
sensors=building_df[0]['sensors']

heat_flow_sensor = []
total_heat_sensor = []
for sensor in sensors:
    if sensor['sensor_id'] == heat_flow_sensor_dict[current_building_name]:
        heat_flow_sensor = sensor
        
    if sensor['sensor_id'] == total_heat_sensor_dict[current_building_name]:
        total_heat_sensor = sensor 


In [7]:
all_buildings = server.buildings()
org_df = pd.DataFrame(all_buildings)

In [8]:
title_md = '''# Heat Flow for {} building'''
title_md = title_md.format(current_building_name)

In [9]:
Markdown(title_md)

# Heat Flow for Quinhagak building

In [10]:
averaging_string = str(averaging_time_length) + 'h'
fifteen_min_averages = server.sensor_readings((heat_flow_sensor_dict[current_building_name], 'heat_flow'),
                                          start_ts = datetime.datetime.now() - relativedelta(years=1),
                                            end_ts = datetime.datetime.now(),
                                          averaging = averaging_string)

In [11]:
# Get rid of any potential erroneous data that is listed as a negative electric usage
fifteen_min_averages = fifteen_min_averages.query("heat_flow >= 0")


In [12]:
fifteen_min_averages = fifteen_min_averages.reset_index()

In [13]:
fifteen_min_averages = fifteen_min_averages.rename(columns={'index':'datetime_col'})

In [14]:
fifteen_min_averages['dayofweek'] = fifteen_min_averages.datetime_col.apply(lambda x: x.dayofweek)

In [15]:
fifteen_min_averages['date'] = fifteen_min_averages.datetime_col.apply(lambda x: x.date())

In [16]:
fifteen_min_averages['time'] = fifteen_min_averages.datetime_col.apply(lambda x: x.time())

In [28]:
current_month_start = fifteen_min_averages.date.iloc[-1] - pd.offsets.Day(31)

In [29]:
current_month = fifteen_min_averages.query("datetime_col >= @current_month_start")

In [ ]:
current_month_w_day_avgs = pd.merge(current_month, day_avgs, how='left',
                                  left_on=['dayofweek', 'time'],
                                  right_on=['dayofweek', 'time'], 
                                  suffixes=('', '_historical_avg'))

In [36]:
this_month = go.Scatter(x = current_month.datetime_col,
                   y = averaging_time_length * current_month.heat_flow,
                   line = dict(color = '#ca0020'),
                   name='Current month heat flow')

layout = dict(title = 'Heat Flow: Current Month',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Heat Flow (KBTU, ' + str(averaging_time_length) + ' hour flow)')
             )

data = [this_month]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [37]:
fuel_per_kbtu= 1/(kbtu_per_gallon_oil*boiler_efficiency)

this_month = go.Scatter(x = current_month.datetime_col,
                   y = fuel_per_kbtu*averaging_time_length * current_month.heat_flow,
                   line = dict(color = '#ca0020'),
                   name='Current Month savings')
layout = dict(title = 'Fuel Saved: Current Month',
              xaxis = dict(title='Date and Time'),
              yaxis = dict(title='Fuel Saved (Gallons, ' + str(averaging_time_length) + ' hour savings)')
             )

data = [this_month]

fig = go.Figure(dict(data=data, layout=layout))

fig.show()

In [38]:
fuel_price_df = get_fuel_df('city-util/proc/city.pkl')
fuel_price_df = fuel_price_df.loc[fuel_price_df['Name'] == current_building_name]
fuel_price=fuel_price_df.iloc[0]['GasPrice']
if (math.isnan(fuel_price)):
    if current_building_name in fuel_price_Dict:
        fuel_price = fuel_price_Dict[current_building_name]
if (not math.isnan(fuel_price)):
    cost_per_kbtu= fuel_price/(kbtu_per_gallon_oil*boiler_efficiency)
    this_month = go.Scatter(x = current_month.datetime_col,
                       y = cost_per_kbtu*averaging_time_length * current_month.heat_flow,
                       line = dict(color = '#ca0020'),
                       name='Current month savings')


    layout = dict(title = 'Heat Recovery Money Saved',
                  xaxis = dict(title='Date and Time'),
                  yaxis = dict(title='Heat Recovery Cost Savings (Dollars, ' + str() + ' hour savings)')
                 )

    data = [this_month]

    fig = go.Figure(dict(data=data, layout=layout))

    fig.show()
